In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

from keras.utils.np_utils import to_categorical
from keras.layers import Activation, add, BatchNormalization, Conv2D, MaxPool2D, Dense, Flatten, Dropout
from keras.models import Sequential
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

import warnings
warnings.filterwarnings('ignore')
# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
train.head()

In [ ]:
train_X, val_X, train_Y, val_Y = train_test_split(train.iloc[:,1:].values, 
                                                train.iloc[:,0].values, test_size=0.1)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12,6))
ax[0].plot(train_X[5])
ax[0].set_title('784x1 data')
ax[1].imshow(train_X[5].reshape(28,28), cmap='gray')
ax[1].set_title('28x28 data')

In [ ]:
# Reshape all data to 28 x 28
train_X, val_X = train_X.reshape(-1, 28, 28, 1), val_X.reshape(-1, 28, 28, 1)
# Convert to 0-1 range
train_X = train_X.astype("float32")/255.0
val_X = val_X.astype("float32")/255.0

In [ ]:
# One-hot encoding
train_Y, val_Y = to_categorical(train_Y), to_categorical(val_Y)

In [ ]:
model = Sequential()
model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation = 'relu', input_shape = (28, 28, 1)))
model.add(BatchNormalization())
model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())

model.add(MaxPool2D(strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())

model.add(MaxPool2D(strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))


In [ ]:
# Data augmentation - from adversarial learning- Jacobian
datagen = ImageDataGenerator(zoom_range = 0.1,
                            height_shift_range = 0.1,
                            width_shift_range = 0.1,
                            rotation_range = 10, horizontal_flip=True)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer = Adam(lr=1e-4), metrics=["accuracy"])

In [ ]:
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)


In [ ]:
hist = model.fit_generator(datagen.flow(train_X, train_Y, batch_size=16),
                           steps_per_epoch=500,
                           epochs=20, #Increase this when not on Kaggle kernel
                           verbose=2,  #1 for ETA, 0 for silent
                           validation_data=(val_X, val_Y), #For speed
                           callbacks=[annealer])

In [ ]:
final_loss, final_acc = model.evaluate(val_X, val_Y)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))



In [ ]:
y_hat = model.predict(val_X)
y_pred = np.argmax(y_hat, axis=1)
y_true = np.argmax(val_Y, axis=1)
cm = confusion_matrix(y_true, y_pred)
print(cm)

In [ ]:
test_X[:1].reshape(28, 28, 1)

In [ ]:
mnist_testset = np.loadtxt('../input/test.csv', skiprows=1, dtype='int', delimiter=',')
test_X = mnist_testset.astype("float32")
test_X = test_X.reshape(-1, 28, 28, 1)/255.
y_hat = model.predict(test_X, batch_size=64)
y_pred = np.argmax(y_hat,axis=1)
y_pred

In [ ]:
with open('submission_keras.csv', 'w') as f :
    f.write('ImageId,Label\n')
    for i in range(len(y_pred)) :
        f.write("".join([str(i+1),',',str(y_pred[i]),'\n']))